In [2]:
import pandas as pd
import numpy as np
import os
import json
import jsonpath

In [3]:
video_info = pd.read_csv('./douyin_ich_video_info.csv')

useful_video_list = [int(x.split('.')[0]) for x in os.listdir('./video_content')]

len(useful_video_list)

len(video_info)

26085

In [4]:
useful_video_info = video_info[video_info['video_id'].isin(useful_video_list)]

useful_video_info.to_csv('./useful_douyin_ich_video_info.csv', index=False)

In [17]:
useful_video_info.columns

Index(['ich_type', 'ich_name', 'video_id', 'video_url', 'video_author',
       'author_verify', 'author_signature', 'author_like', 'video_title',
       'video_tag', 'video_duration', 'num_com', 'num_like', 'num_share',
       'size', 'province', 'content'],
      dtype='object')

In [5]:
# useful_video_info.groupby('ich_type').agg({'video_id':'count'}).rename(columns={'id':'flow'}).reset_index()
useful_video_info.groupby('ich_type').agg({'video_id':'count'}).rename(columns={'video_id':'each_type_count'}).reset_index()

,ich_type,each_type_count
0,传统戏剧,4903
1,传统美术,5980
2,传统舞蹈,6225
3,传统音乐,6076
4,曲艺,2664


In [6]:
useful_video_info['video_id'].unique().shape

(25848,)

In [5]:
import traceback
plus_data = []
for video_id in useful_video_info['video_id'].unique().tolist():
    try:
        ich_name = useful_video_info[useful_video_info['video_id']==video_id]['ich_name'].values[0]
        with open('./video_info_json/info_{}_{}.json'.format(ich_name,video_id), 'r', encoding='utf-8') as f:
            video_info = json.load(f)
        author_age = jsonpath.jsonpath(video_info, '$..author.user_age')[0]
        video_quality_list = jsonpath.jsonpath(video_info, '$..video.bit_rate[*].gear_name')
        video_bit_rate_list = jsonpath.jsonpath(video_info, '$..video.bit_rate[*].bit_rate')
        plus_data.append({'video_id':video_id, 'author_age':author_age, 'video_quality_list':video_quality_list, 'video_bit_rate_list':video_bit_rate_list})
    except:
        traceback.print_exc()
        print(video_id)

In [6]:
plus_data

[{'video_id': 6866264054491663620,
  'author_age': 64,
  'video_quality_list': ['normal_480_0'],
  'video_bit_rate_list': [776958]},
 {'video_id': 6885629306484133135,
  'author_age': -1,
  'video_quality_list': ['adapt_lowest_720_1', 'adapt_540_1'],
  'video_bit_rate_list': [853044, 771216]},
 {'video_id': 6906839221604994317,
  'author_age': 49,
  'video_quality_list': ['normal_540_0'],
  'video_bit_rate_list': [1805717]},
 {'video_id': 6950577893105880335,
  'author_age': 64,
  'video_quality_list': ['normal_720_0'],
  'video_bit_rate_list': [785353]},
 {'video_id': 6975071420242021647,
  'author_age': -1,
  'video_quality_list': ['normal_360_0'],
  'video_bit_rate_list': [644312]},
 {'video_id': 6984051093340802341,
  'author_age': -1,
  'video_quality_list': ['adapt_lowest_720_1', 'adapt_540_1'],
  'video_bit_rate_list': [847391, 780903]},
 {'video_id': 6986773192723483935,
  'author_age': -1,
  'video_quality_list': ['normal_720_0'],
  'video_bit_rate_list': [618768]},
 {'video_i

In [7]:
plus_data_df = pd.DataFrame(plus_data)

plus_useful_video_info = useful_video_info.merge(plus_data_df, on='video_id', how='left')

plus_useful_video_info

,ich_type,ich_name,video_id,video_url,video_author,author_verify,author_signature,author_like,video_title,video_tag,video_duration,num_com,num_like,num_share,size,province,content,author_age,video_quality_list,video_bit_rate_list
0,传统戏剧,一勾勾,6866264054491663620,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,NaN,爱好戏曲，特好京剧！,7150,我们又排地方戏(一勾勾)墙头记！我还是扮演那受气老头！,False,152.195,15,29,1,14781146,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...,64,[normal_480_0],[776958]
1,传统戏剧,一勾勾,6885629306484133135,https://api.amemv.com/aweme/v1/play/?video_id=...,临邑县融媒体中心,临邑县融媒体中心官方抖音号,新闻前沿 百姓身边\n权威发布 热点资讯\n投稿加V:lyxrmtzx,347737,#非物质文化遗产 一勾勾，#临邑 的传统剧种。,"['非物质文化遗产', '临邑']",17.995,12,160,18,1918818,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...,-1,"[adapt_lowest_720_1, adapt_540_1]","[853044, 771216]"
2,传统戏剧,一勾勾,6906839221604994317,https://api.amemv.com/aweme/v1/play/?video_id=...,泉正甜,NaN,做快乐的人，越变越年轻！歌颂祖国，歌颂人民，歌颂生活，歌颂爱情，歌颂生命，关注人间疾苦，解说...,12930,糅合了吕剧豫剧秦腔河北梆子的临邑戏曲文化遗产——一勾勾挺好听。,False,73.834,0,7,0,16665420,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...,49,[normal_540_0],[1805717]
3,传统戏剧,一勾勾,6950577893105880335,https://api.amemv.com/aweme/v1/play/?video_id=...,戏曲票友吴善家,NaN,爱好戏曲，特好京剧！,7150,我们的地方戏，国家级非遗，《一勾勾》剧又排完一出戏…,False,65.063,9,34,2,6387185,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...,64,[normal_720_0],[785353]
4,传统戏剧,一勾勾,6975071420242021647,https://api.amemv.com/aweme/v1/play/?video_id=...,居士,NaN,NaN,340,国家非遗（一勾勾），复排“王汉喜借年”。,False,312.313,14,30,0,25153379,山东省临邑县,“一勾勾”是由高唐一带的鼓子秧歌衍化而成的地方剧种。高唐鼓子秧歌是一种民间演唱艺术，演唱...,-1,[normal_360_0],[644312]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25843,曲艺,龙舟说唱,7162006074617302309,https://api.amemv.com/aweme/v1/play/?video_id=...,粤风堂,店铺账号,弘扬优秀岭南文化\n汇聚众多非遗项目\n探寻新奇有趣故事\n感受名家大师魅力,328,国家级非遗龙舟说唱未来会消失吗？#传统文化 #弘扬和传承民族文化 #非遗传承 #民俗文化 #...,"['传统文化', '弘扬和传承民族文化', '非遗传承', '民俗文化', '传递正能量',...",91.439,0,4,1,7890263,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...,-1,[normal_720_0],[690319]
25844,曲艺,龙舟说唱,7163939438458916135,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,NaN,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨学好本领建中华 #文艺汇演 #演出现场 #活动现场 #顺德龙舟说唱,"['文艺汇演', '演出现场', '活动现场', '顺德龙舟说唱']",460.406,0,0,0,57012293,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...,-1,[normal_720_0],[990643]
25845,曲艺,龙舟说唱,7163939831742008614,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,NaN,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨守法欢乐渡假期 #文艺汇演 #文化旅游 #演出现场 #振兴乡村 #顺德龙舟说唱,"['文艺汇演', '文化旅游', '演出现场', '振兴乡村', '顺德龙舟说唱']",128.366,0,0,0,13133944,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...,-1,[normal_720_0],[818524]
25846,曲艺,龙舟说唱,7163940216791780645,https://api.amemv.com/aweme/v1/play/?video_id=...,海韵教育,NaN,教育就是一棵树摇动一棵树，一朵云推动一朵云，一个灵魂唤醒一个灵魂！,9956,顺德龙舟说唱丨战疫情报民安 #振兴乡村 #活动现场 #文艺汇演 #顺德龙舟说唱,"['振兴乡村', '活动现场', '文艺汇演', '顺德龙舟说唱']",226.134,0,0,0,26289182,广东省佛山市顺德区,龙舟歌在民间又称“唱龙舟”或简称“龙舟”，是流行于广东珠江三角洲地区的一种曲艺形式，一般...,-1,[normal_720_0],[930039]


In [8]:
plus_useful_video_info.to_csv('./plus_useful_douyin_ich_video_info.csv', index=False)

In [12]:
len(plus_useful_video_info[plus_useful_video_info['author_verify'].isnull()])/len(plus_useful_video_info)

0.7498452491488703

+ 过滤 排名靠前的非ICH相关视频内容
+ 考虑怎样获取 视频作者的账号是否是专门搞他这个ICH的作者
+ 可视化出来每个视频作者的地区分布（农村，城市）

In [20]:
plus_useful_video_info = plus_useful_video_info[plus_useful_video_info['ich_name']!='舞']
plus_useful_video_info.sort_values(['num_like','num_com','num_share'], ascending=False).head(10)

,ich_type,ich_name,video_id,video_url,video_author,author_verify,author_signature,author_like,video_title,video_tag,video_duration,num_com,num_like,num_share,size,province,content,author_age,video_quality_list,video_bit_rate_list
20907,传统戏剧,豫剧,6800558482698407182,https://api.amemv.com/aweme/v1/play/?video_id=...,湖北新闻,湖北广播电视台新闻中心官方抖音号,📺新无止境 \n🎙闻达天下\n联系邮箱：<3450849514@qq.com>,1518216202,大爷不配合治疗，#河南护士一曲#豫剧给大爷震住了！ #房里有戏,"['河南', '豫剧', '房里有戏']",35.290,143510,2885442,25023,2191500,河南省,豫剧也叫河南梆子、河南高调、河南讴，豫西山区则称之为靠山吼。它是我国梆子声腔剧种中极为重...,3,"[adapt_lowest_720_1, adapt_540_1]","[496797, 409861]"
21182,传统戏剧,赣剧,7115027495509888256,https://api.amemv.com/aweme/v1/play/?video_id=...,赣剧张钰,NaN,国家二级演员。师承著名赣剧表演艺术家胡瑞华。\n江西省赣剧院演员\n主演剧目：《红星恋歌》《...,5698925,1分多钟，默契十足的抢装❤️#戏曲也抖音 #传承文化 #赣剧红楼梦,"['戏曲也抖音', '传承文化', '赣剧红楼梦']",61.834,75396,2767670,75218,10013145,江西省赣剧院,赣剧，江西省地方剧种。起源于赣东北，流行于江西省境内。其声腔基础为明代四大声腔之一的弋阳...,35,"[adapt_lowest_720_1, adapt_540_1]","[1295487, 1075510]"
15649,曲艺,相声,6675727664692710668,https://api.amemv.com/aweme/v1/play/?video_id=...,我的偶像巨顽皮,娱乐视频自媒体,关注我别错过，德云社/薛之谦👉合集\n\n商务合作：abc1101581108\n\n如有侵...,1206574530,弘扬传统文化，说相声得从娃娃抓起🤣#德云社 #秦霄贤 #孟鹤堂,"['德云社', '秦霄贤', '孟鹤堂']",36.433,23556,2729298,65085,2973883,中国广播艺术团,相声是普及面最广、最受群众欢迎的曲艺品种之一，以滑稽、讽刺见长，充满戏剧性。它大约在清代...,19,"[adapt_540_0, normal_720_0, normal_540_0]","[651774, 441834, 383686]"
13795,传统舞蹈,热巴舞,6712785116403354887,https://api.amemv.com/aweme/v1/play/?video_id=...,水冰月,NaN,🈴️：yingx177（备注品牌）,84836349,学了女神#迪丽热巴 的一首欢快的舞蹈，整个人都跳开心了。,['迪丽热巴'],10.566,56240,2574596,126814,2027968,云南省迪庆藏族自治州,None,23,"[adapt_540_0, normal_540_0]","[1535466, 1514448]"
23419,传统音乐,阿里郎,6985062966228012328,https://api.amemv.com/aweme/v1/play/?video_id=...,金润吉,NaN,代表作：\n《Tell me》《Love Love》《学到老 爱到老》《Disco》《再次爱...,22586641,Just tell me ～为什么～眼神是不是有话要对我说？,False,28.397,83110,2444102,169862,1566157,吉林省延边朝鲜族自治州,《阿里郎打令》是中国朝鲜族抒情民歌的代表性作品，在中国朝鲜族群众中具有极大的影响力。阿里...,-1,"[adapt_lowest_1080_1, adapt_lowest_720_1, adap...","[441217, 320137, 311084]"
15651,曲艺,相声,6785874034421714179,https://api.amemv.com/aweme/v1/play/?video_id=...,东方卫视,东方卫视,每晚19:30电视剧《县委大院》\n周三22:00《梦想改造家》\n周五22:30《家有姐妹...,909676050,听说#岳云鹏 的年终奖了吗？提起逗哏捧哏的排行榜#孙越 生气了？#东方卫视春晚,"['岳云鹏', '孙越', '东方卫视春晚']",127.407,22847,2442451,11063,8910324,中国广播艺术团,相声是普及面最广、最受群众欢迎的曲艺品种之一，以滑稽、讽刺见长，充满戏剧性。它大约在清代...,4,[adapt_lowest_720_1],[559487]
7364,传统音乐,广东音乐,6886403110084873485,https://api.amemv.com/aweme/v1/play/?video_id=...,小鸭子老师,NaN,此账号是小鸭子老师唯一抖音号\n请勿相信其他账号\n🐧音乐搜索：小鸭子老师 \n每周五周六晚...,50762998,小家伙现场叫我唱粤语歌,False,40.867,44765,2284155,32822,6960795,广东省广州市,广东音乐是流行于以广州为中心的珠江三角洲及广府方言区的传统丝竹乐种，是岭南民间传统文化的...,31,"[adapt_lowest_720_1, adapt_540_1]","[1362624, 1310523]"
7720,曲艺,弹唱,6808126743140945163,https://api.amemv.com/aweme/v1/play/?video_id=...,Zkaaai,NaN,单身勿cue 单身勿cue\nwy☁️/q因/👖狗/🧣/🍠：zkaaai\n合作联系：zka...,51796517,#弹唱 没有想过是你和我一起深陷其中#wonderfulu,"['弹唱', 'wonderfulu']",27.375,86232,2275319,72262,2218005,山西省吕梁市离石区,None,23,"[adapt_lowest_720_1, adapt_540_1]","[648184, 636164]"
13796,传统舞蹈,热巴舞,6715269696582601988,https://api.amemv.com/aweme/v1/play/?video_id=...,线条君LineDancer,NaN,线条牌跳舞机～合拍会火～＠我点❤️\n🤝\/ ✉️：drivervictoria,103288048,#迪丽热巴 这个舞！根本停不下来！,['迪丽热巴'],11.309,33637,2271434,113489,3172735,云南省迪庆藏族自治州,None,4,"[normal_540_0, adapt_540_0]","[2244396, 1703192]"
7374,传统音乐,广东音乐,7093077180690779406,https://api.amemv.com/aweme/v1/play/?video_id=...,陈艺鹏（粤曲腔）,NaN,商务联系v：kenpochen\n歌曲合作、短视频合作、广告合作\n全网同名，听歌搜索名字🔍...,9931324,《声声慢》越听越上头，是老了吗，竟然爱上这腔调。#国语VS粤语 #创作灵感 #陈艺鹏 #粤剧...,"['国语vs粤语', '创作灵感', '陈艺鹏', '粤剧', '粤曲小调', '声声慢',...",62.368,96955,2177925,194727,3875575,广东省广州市,广东音乐是流行于以广州为中心的珠江三角洲及广府方言区的传统丝竹乐种，是岭南民间传统文化的...,-1,"[adapt_lowest_720_1, adapt_540_1, normal_480_0]","[497123, 435703, 347720]"


In [25]:
print(len(plus_useful_video_info['ich_name'].unique()))
print(len(plus_useful_video_info[plus_useful_video_info['ich_name'].str.contains('族')]['ich_name'].unique()))

1498
224


In [28]:
minor_list = '汉族、蒙古族、回族、藏族、维吾尔族、苗族、彝族、壮族、布依族、朝鲜族、满族、侗族、瑶族、白族、土家族、哈尼族、哈萨克族、傣族、黎族、僳僳族、佤族、畲族、高山族、拉祜族、水族、东乡族、纳西族、景颇族、柯尔克孜族、土族、达斡尔族、仫佬族、羌族、布朗族、撒拉族、毛南族、仡佬族、锡伯族、阿昌族、普米族、塔吉克族、怒族、乌孜别克族、俄罗斯族、鄂温克族、德昂族、保安族、裕固族、京族、塔塔尔族、独龙族、鄂伦春族、赫哲族、门巴族、珞巴族、基诺族'.split('、')

In [40]:
c=0
for minor in minor_list[1:]:
    c += len(plus_useful_video_info.loc[(plus_useful_video_info['ich_name'].str.contains(minor[:-1]))])
c

6225

In [33]:
c/len(plus_useful_video_info)

0.24107350321431337

In [2]:
from geopy.geocoders import Nominatim

address = "广州"
gps = Nominatim()
location = gps.geocode(address)
print (location.longitude,location.latitude)


ConfigurationError: Using Nominatim with default or sample `user_agent` "geopy/2.3.0" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.